In [1]:
import numpy as np

In [2]:
def moisture_flux_convergence(u, v, q, rho, dx, dy):
    
    # Initialize 2D arrays for derivatives
    dudx = np.zeros_like(u)
    dvdy = np.zeros_like(v)
    dqdx = np.zeros_like(q)
    dqdy = np.zeros_like(q)
    
    # Central differences for interior points
    
    # dudx[:, 1:-1]: all rows, all columns except first and last 
    # u[:, 2:]: all rows, starts at 3rd column
    # u[:, :-2]: all rows, all columns except last two 
    dudx[:, 1:-1] = (u[:, 2:] - u[:, :-2]) / (2 * dx)
    
    # dvdy[1:-1, :]: all rows except first and last, all columns
    # v[2:, :]: starts at third row, all columns
    # v[:-2, :]: all rows except last two, all columns
    dvdy[1:-1, :] = (v[2:, :] - v[:-2, :]) / (2 * dy)
    
    dqdx[:, 1:-1] = (q[:, 2:] - q[:, :-2]) / (2 * dx)
    dqdy[1:-1, :] = (q[2:, :] - q[:-2, :]) / (2 * dy)
    
    # Handling boundary points with one-sided differences:
    # Left and right boundaries (x-direction)
    dudx[:, 0] = (u[:, 1] - u[:, 0]) / dx       # forward difference at the left boundary
    dudx[:, -1] = (u[:, -1] - u[:, -2]) / dx    # backward difference at the right boundary
    
    dqdx[:, 0] = (q[:, 1] - q[:, 0]) / dx
    dqdx[:, -1] = (q[:, -1] - q[:, -2]) / dx
    
    # Top and bottom boundaries (y-direction)
    dvdy[0, :] = (v[1, :] - v[0, :]) / dy       # forward difference at the top boundary
    dvdy[-1, :] = (v[-1, :] - v[-2, :]) / dy    # backward difference at the bottom boundary
    
    dqdy[0, :] = (q[1, :] - q[0, :]) / dy
    dqdy[-1, :] = (q[-1, :] - q[-2, :]) / dy
    
    # Moisture advection
    moisture_advection = -(u * dqdx + v * dqdy) / rho
    
    # Dynamical convergence
    dynamical_convergence = -q * (dudx + dvdy) / rho
    
    # Total moisture flux convergence
    mfc = dynamical_convergence + moisture_advection
    
    # Units --> m^3/(kg*s)
    
    return mfc, dynamical_convergence, moisture_advection



# Vertically integrated moisture flux convergence and its components
def vertically_integrated_mfc(u_levels, v_levels, q_levels, temperature_levels, dx, dy, pressure_levels):
    """
    u_levels, v_levels, q_levels: 3D arrays (pressure, lat, lon) of wind and humidity at different pressure levels
    dx, dy: grid spacing in meters (uniform for simplicity)
    pressure_levels: 1D array of pressure levels in Pascals (Pa), typically ordered from top to surface
    """
    g = 9.81  # gravitational acceleration in m/s^2
    rho_water = 1000 # kg/m^3
    
    # Initialize arrays to store MFC, dynamical convergence, and moisture advection at each pressure level
    mfc_levels = np.zeros_like(u_levels, dtype=np.float64)
    dyn_conv_levels = np.zeros_like(u_levels, dtype=np.float64)
    moist_adv_levels = np.zeros_like(u_levels, dtype=np.float64)
    rho_air_levels = np.zeros_like(u_levels, dtype=np.float64)
    
    # Loop over pressure levels and compute MFC and its components at each level
    for i in range(len(pressure_levels)):
        pressure = pressure_levels[i] # single value
        q = q_levels[i]  # specific humidity at the current level (10x10)
        T = temperature_levels[i]  # temperature at the current level (10x10)
        
        # Calculate rho_air using the formula for moist air (rho = pressure/Rd*Tv, where Tv = (1 + 0.608)T)
        rho_air_levels[i] = pressure / (Rd * (1 + 0.608 * q) * T)
        mfc_levels[i], dyn_conv_levels[i], moist_adv_levels[i] = moisture_flux_convergence(u_levels[i], v_levels[i], q_levels[i], rho_air_levels[i], dx, dy)
    
    # Use trapezoidal rule to integrate MFC and its components over pressure
    dp = np.diff(pressure_levels)  # Differences between pressure levels (positive when integrating from top to surface)
    
    # The integrations (for full column) are 2D
    mfc_integrated = np.zeros_like(mfc_levels[0], dtype=np.float64)        # Vertically integrated total MFC
    dyn_conv_integrated = np.zeros_like(dyn_conv_levels[0], dtype=np.float64)  # Vertically integrated dynamical convergence
    moist_adv_integrated = np.zeros_like(moist_adv_levels[0], dtype=np.float64)  # Vertically integrated moisture advection
    
    # Trapezoidal integration: sum over pressure levels
    for i in range(len(dp)):
        mfc_integrated += 0.5 * (mfc_levels[i] + mfc_levels[i+1]) * (dp[i])
        dyn_conv_integrated += 0.5 * (dyn_conv_levels[i] + dyn_conv_levels[i+1]) * (dp[i])
        moist_adv_integrated += 0.5 * (moist_adv_levels[i] + moist_adv_levels[i+1]) * (dp[i])
    
    # Convert to vertically integrated MFC and its components by dividing by g
    mfc_integrated = mfc_integrated * (rho_water / g)
    dyn_conv_integrated = dyn_conv_integrated * (rho_water / g)
    moist_adv_integrated = moist_adv_integrated * (rho_water / g)
    
    # Units --> kg/(m2*s)
    return mfc_integrated, dyn_conv_integrated, moist_adv_integrated



In [3]:
num_levels = 5  # Number of pressure levels
grid_size = 10  # 10x10 grid
Rd = 287 # J/kg·K 

# Wind (u) profiles, m/s
u_levels = np.array([
    np.random.uniform(20, 30, (grid_size, grid_size)),  # u at level 1 (500 hPa)
    np.random.uniform(15, 25, (grid_size, grid_size)),  # u at level 2 (700 hPa)
    np.random.uniform(10, 20, (grid_size, grid_size)),  # u at level 3 (850 hPa)
    np.random.uniform(5, 15, (grid_size, grid_size)),   # u at level 4 (900 hPa)
    np.random.uniform(0, 10, (grid_size, grid_size))    # u at level 5 (1000 hPa)
])

# Wind (v) profiles, m/s
v_levels = np.array([
    np.random.uniform(10, 20, (grid_size, grid_size)),  # v at level 1 (500 hPa)
    np.random.uniform(5, 15, (grid_size, grid_size)),   # v at level 2 (700 hPa)
    np.random.uniform(5, 10, (grid_size, grid_size)),   # v at level 3 (850 hPa)
    np.random.uniform(2, 8, (grid_size, grid_size)),    # v at level 4 (900 hPa)
    np.random.uniform(0, 5, (grid_size, grid_size))     # v at level 5 (1000 hPa)
])

# Specific humidity (q) profiles, kg/kg
q_levels = np.array([
    np.random.uniform(0.001, 0.003, (grid_size, grid_size)),  # q at level 1 (500 hPa)
    np.random.uniform(0.002, 0.005, (grid_size, grid_size)),  # q at level 2 (700 hPa)
    np.random.uniform(0.005, 0.010, (grid_size, grid_size)),  # q at level 3 (850 hPa)
    np.random.uniform(0.007, 0.015, (grid_size, grid_size)),  # q at level 4 (900 hPa)
    np.random.uniform(0.010, 0.020, (grid_size, grid_size))   # q at level 5 (1000 hPa)
])

# Temperature profiles, Kelvin
temperature_levels = np.array([
    np.random.uniform(230, 250, (grid_size, grid_size)),  # Temperature at level 1 (500 hPa)
    np.random.uniform(250, 270, (grid_size, grid_size)),  # Temperature at level 2 (700 hPa)
    np.random.uniform(260, 280, (grid_size, grid_size)),  # Temperature at level 3 (850 hPa)
    np.random.uniform(270, 290, (grid_size, grid_size)),  # Temperature at level 4 (900 hPa)
    np.random.uniform(280, 300, (grid_size, grid_size))   # Temperature at level 5 (1000 hPa)
])

# Sample data for pressure levels, Pascals
pressure_levels = np.array([50000, 70000, 85000, 90000, 100000]) # Pressure levels in Pascals

# Grid spacing in meters (simplified)
dx =  20000 # grid spacing in longitude-direction (meters)
dy = 27750  # grid spacing in latitude-direction (meters)

# Calculate vertically integrated MFC, dynamical convergence, and moisture advection
mfc_integrated, dyn_conv_integrated, moist_adv_integrated = vertically_integrated_mfc(u_levels, v_levels, q_levels, temperature_levels, dx, dy, pressure_levels)

# Output the results
print("Vertically Integrated Moisture Flux Convergence:")
print(np.max(mfc_integrated), np.min(mfc_integrated))
print("")
print("Vertically Integrated Dynamical Convergence:")
print(np.max(dyn_conv_integrated), np.min(dyn_conv_integrated))
print("")
print("Vertically Integrated Moisture Advection:")
print(np.max(moist_adv_integrated), np.min(moist_adv_integrated))


Vertically Integrated Moisture Flux Convergence:
7.984369320704783 -10.2304884889354

Vertically Integrated Dynamical Convergence:
4.041490698335598 -6.410167562649102

Vertically Integrated Moisture Advection:
5.80567173630881 -5.764415393825146
